<a href="https://colab.research.google.com/github/tcalderon-space/COMP553/blob/main/CS533_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tasks

  - Decide BERT Model
    - 20 different ver.
 - GLUE Task (RTE)
 - Write Report


# Rubric Instructions (Canvas):
Link:


# On BERT Github:
- Predicting Movie Reviews with BERT on TF Hub.ipynb
  - Tutorial: https://colab.research.google.com/github/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb
  - 1st Attempt w/ Movie Example
     Link: https://colab.research.google.com/drive/144qVBZjWonTjGnwBgpIpZcwEQzv0PeRe?authuser=1#scrollTo=W7SxepqjxnHh

  - Movie Example with RTE testing :
  https://colab.research.google.com/drive/190_ipyuJ60TwkhUpyVTuLriSE-qnaw9R?usp=sharing

- Updated w/ testing on RTE
  -uses tensorflow/text to train then implements RTE to do testing + accuracy
https://colab.research.google.com/drive/1BWKo44WPVJIxxluhuLdtrqSUvkpF-XQd?usp=sharing


Link for fine-tuning (printing out the accuracy of GLUE)
https://www.tensorflow.org/tfmodels/nlp/fine_tune_bert#build_train_and_export_the_model

# Install + Import Libraries

Install Tensor flow + Bert
Install transformers

In [ ]:
!pip install transformers
!pip install bert-tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 2.5 MB/s eta 0:00:00


In [ ]:
from transformers import TFBertForSequenceClassification, BertTokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

# Tokenizer



In [ ]:
# Define the name of the pre-trained BERT model
model_name = 'bert-base-uncased'

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name)

# Load the pre-trained BERT model for sequence classification, disabling safetensors
num_labels = len(label_list)
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, use_safetensors=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

NameError: name 'label_list' is not defined

# Prepare Data

In [ ]:
import os
import re
import pandas as pd
import tensorflow as tf
import tarfile

# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with open(os.path.join(directory, file_path), "r", encoding="utf-8") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match(r"\d+_(\d+)\.txt", file_path).group(1)) # Added raw string
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset_tar_path = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz",
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
      extract=False) # Do not extract here

  dataset_dir = os.path.join(os.path.dirname(dataset_tar_path), "aclImdb")

  # Extract the tar file manually
  if not os.path.exists(dataset_dir):
      print("Extracting dataset...")
      with tarfile.open(dataset_tar_path, 'r:gz') as tar:
          tar.extractall(path=os.path.dirname(dataset_tar_path))


  train_df = load_dataset(os.path.join(dataset_dir, "train"))
  test_df = load_dataset(os.path.join(dataset_dir, "test"))

  return train_df, test_df

train, test = download_and_load_datasets()

# To keep training fast, we'll take a sample of 5000 train and test examples, respectively.
train = train.sample(5000)
test = test.sample(5000)

DATA_COLUMN = 'sentence'
LABEL_COLUMN = 'polarity'

# Use the InputExample class from BERT's run_classifier code to create examples from the data
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
        Args:
          guid: Unique id for the example.
          text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
          text_b: (Optional) string. The untokenized text of the second sequence.
            Only used for sequence pair tasks.
          label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN],
                                                                   text_b = None,
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: InputExample(guid=None,
                                                                   text_a = x[DATA_COLUMN],
                                                                   text_b = None,
                                                                   label = x[LABEL_COLUMN]), axis = 1)

# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128

def convert_examples_to_features(examples, tokenizer, max_seq_length):
    features = []
    for (ex_index, example) in enumerate(examples):
        inputs = tokenizer(
            text=example.text_a,
            text_pair=example.text_b,
            padding='max_length',
            truncation=True,
            max_length=max_seq_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_tensors='tf' # Return TensorFlow tensors
        )

        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]
        token_type_ids = inputs['token_type_ids'][0]
        label = example.label

        features.append({
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
            'label': label,
        })
    return features

# Apply the conversion function to the training and test data
train_features = convert_examples_to_features(train_InputExamples, tokenizer, MAX_SEQ_LENGTH)
test_features = convert_examples_to_features(test_InputExamples, tokenizer, MAX_SEQ_LENGTH)

84125825/84125825 [==============================] - 7s 0us/step
Extracting dataset...


/tmp/ipython-input-3432139150.py:39: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=os.path.dirname(dataset_tar_path))
TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


# Create tensorflow dataset

In [ ]:
import tensorflow as tf

# Define BATCH_SIZE
BATCH_SIZE = 32

# Create a TensorFlow Dataset from the training features
train_dataset = tf.data.Dataset.from_tensor_slices({
    'input_ids': [f['input_ids'] for f in train_features],
    'attention_mask': [f['attention_mask'] for f in train_features],
    'token_type_ids': [f['token_type_ids'] for f in train_features],
    'label': [f['label'] for f in train_features]
})

# Shuffle the training dataset
train_dataset = train_dataset.shuffle(len(train_features))

# Batch the training dataset
train_dataset = train_dataset.batch(BATCH_SIZE)

# Create a TensorFlow Dataset from the test features
test_dataset = tf.data.Dataset.from_tensor_slices({
    'input_ids': [f['input_ids'] for f in test_features],
    'attention_mask': [f['attention_mask'] for f in test_features],
    'token_type_ids': [f['token_type_ids'] for f in test_features],
    'label': [f['label'] for f in test_features]
})

# Batch the test dataset
test_dataset = test_dataset.batch(BATCH_SIZE)

print("Train dataset created and batched.")
print("Test dataset created and batched.")

# Build and Compile Model
Build a new model using the loaded BERT model as the base and add a classification layer on top. Compile the model with an appropriate optimizer and loss function.

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.metrics.SparseCategoricalAccuracy()]
)

#Evaluate the model
Evaluate the trained model on the test dataset.

In [ ]:
print("Evaluating the model on the test dataset.")
model.evaluate(test_dataset)

# Make predictions
Adapt the prediction function to use the new model and tokenizer to predict sentiment on new sentences.

In [ ]:
def getPrediction(in_sentences):
    labels = ["Negative", "Positive"]
    # Tokenize the input sentences
    inputs = tokenizer(
        text=in_sentences,
        padding='max_length',
        truncation=True,
        max_length=MAX_SEQ_LENGTH,
        return_attention_mask=True,
        return_token_type_ids=True,
        return_tensors='tf' # Return TensorFlow tensors
    )

    # Get predictions from the model
    # The model expects input_ids, attention_mask, and token_type_ids as keyword arguments
    outputs = model(inputs)

    # The output logits are in outputs.logits
    logits = outputs.logits

    # Get predicted labels (index with the highest logit)
    predicted_labels_indices = tf.argmax(logits, axis=1).numpy()

    # Get probabilities by applying softmax to logits
    probabilities = tf.nn.softmax(logits, axis=1).numpy()

    results = []
    for i, sentence in enumerate(in_sentences):
        result = (sentence, probabilities[i], labels[predicted_labels_indices[i]])
        results.append(result)

    return results

# Define sample sentences for prediction
pred_sentences = [
  "That movie was absolutely awful",
  "The acting was a bit lacking",
  "The film was creative and surprising",
  "Absolutely fantastic!"
]

# Get predictions for the sample sentences
predictions = getPrediction(pred_sentences)

# Print the predictions
print(predictions)